# Imports

In [1]:
from pathlib import Path
import pickle
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt

from applefy.detections.contrast import Contrast
from applefy.utils.positions import center_subpixel

from applefy.utils.file_handling import load_adi_data
from applefy.utils import flux_ratio2mag, mag2flux_ratio

# Load the data

In [2]:
dataset_name = "HD22049_303_199_C-0065_C_"

In [3]:
dataset_file = Path("/fast/mbonse/s4/30_data/" + dataset_name + ".hdf5")
experiment_root_dir = Path("/fast/mbonse/s4/70_results/07_partial_contrast_grid/")

In [4]:
# we need the psf template for contrast calculation
science_data, angles, raw_psf_template_data = load_adi_data(
    dataset_file,
    data_tag="object",
    psf_template_tag="psf_template",
    para_tag="header_object/PARANG")

dit_psf_template = 0.0042560
dit_science = 0.08
fwhm = 3.6

science_data = science_data[:, 12:-12, 12:-12]

psf_template = np.median(raw_psf_template_data, axis=0)
psf_template = psf_template - np.min(psf_template)

# Create the full contrast grid config files

In [5]:
tmp_exp_root = experiment_root_dir / dataset_file.name[:-5]
tmp_exp_root.mkdir(exist_ok=True)

In [6]:
contrast_instance = Contrast(
    science_sequence=science_data,
    psf_template=psf_template,
    parang_rad=angles,
    psf_fwhm_radius=fwhm / 2,
    dit_psf_template=dit_psf_template,
    dit_science=dit_science,
    scaling_factor=1., # A factor to account e.g. for ND filters
    checkpoint_dir=tmp_exp_root)

## Design fake planet experiments

In [7]:
# fake planet brightness
flux_ratios_mag = np.linspace(5., 13, 17)
flux_ratios = mag2flux_ratio(flux_ratios_mag)

print("Brightness of fake planets in mag: " + str(flux_ratios_mag))
print("Planet-to-star flux ratio: " + str(flux_ratios))

Brightness of fake planets in mag: [ 5.   5.5  6.   6.5  7.   7.5  8.   8.5  9.   9.5 10.  10.5 11.  11.5
 12.  12.5 13. ]
Planet-to-star flux ratio: [1.00000000e-02 6.30957344e-03 3.98107171e-03 2.51188643e-03
 1.58489319e-03 1.00000000e-03 6.30957344e-04 3.98107171e-04
 2.51188643e-04 1.58489319e-04 1.00000000e-04 6.30957344e-05
 3.98107171e-05 2.51188643e-05 1.58489319e-05 1.00000000e-05
 6.30957344e-06]


In [8]:
center = center_subpixel(science_data[0])
separations = np.arange(fwhm, fwhm*6.5, fwhm / 2)[1:]
separations/3.6

array([1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5, 5. , 5.5, 6. ])

In [9]:
num_fake_planets = 3

contrast_instance.design_fake_planet_experiments(
    flux_ratios=flux_ratios,
    num_planets=num_fake_planets,
    separations=separations,
    overwrite=True)

Overwriting existing config files.


# Load results of analytical contrast curve and select

In [10]:
contrast_curves = pd.read_pickle("../../contrast_curves/HD22049_303_199_C-0065_C__cc.pkl")

In [11]:
pca_contrast_curves = contrast_curves.drop(["cADI", "s4_mean", "s4_median"], axis=1)
s4_contrast_curve = contrast_curves["s4_mean"]
s4_contrast_curve = flux_ratio2mag(s4_contrast_curve.droplevel(1).iloc[1:11])

# compute the overall best contrast curve
overall_best_pca = np.min(pca_contrast_curves, axis=1)
overall_best_pca = flux_ratio2mag(overall_best_pca.droplevel(1).iloc[1:11])

# Filter for experiments in the range of interest

In [62]:
upper_limits = np.floor(overall_best_pca * 2) / 2 - 0.5
upper_limits[[1.5, 2.0, 2.5]] -= 1.0
upper_limits

separation [$FWHM$]
1.5     4.5
2.0     5.5
2.5     6.0
3.0     7.5
3.5     8.5
4.0     9.0
4.5     9.5
5.0     9.5
5.5    10.0
6.0    10.5
dtype: float64

In [63]:
# used to calculate more missing residuals
#upper_limits = np.ceil(s4_contrast_curve * 2) / 2 + 1.0
#upper_limits

In [64]:
lower_limits = np.ceil(s4_contrast_curve * 2) / 2 + 1.0
lower_limits[[1.5, 2.0, 2.5, 3.0]] += 0.5
lower_limits[[1.5, 2.0]] += 0.5
lower_limits

separation [$FWHM$]
1.5    10.0
2.0    11.0
2.5    11.5
3.0    12.0
3.5    12.0
4.0    12.5
4.5    13.0
5.0    13.0
5.5    13.5
6.0    13.5
Name: s4_mean, dtype: float64

In [65]:
selection_file = "/fast/mbonse/s4/70_results/07_partial_contrast_grid/01_selection_HD22049_303_199_C-0065_C.txt"
#selection_file = "/fast/mbonse/s4/70_results/07_partial_contrast_grid/02_selection_HD22049_303_199_C-0065_C.txt"
don_t_use = []
do_use = []

with open(selection_file, "w") as text_file:
    num_exp = 0
    for tmp_key, values in contrast_instance.experimental_setups.items():

        if tmp_key == "0000":
            text_file.write(dataset_name + " " + tmp_key + "\n")
            num_exp+=1
            do_use.append(tmp_key)
            continue

        tmp_separation = np.round(values["separation"] / 3.6, 1)
        tmp_flux_ratio = np.round(flux_ratio2mag(values["flux_ratio"]), 1)

        within_upper_limit = upper_limits[tmp_separation] <= tmp_flux_ratio
        within_lower_limit = lower_limits[tmp_separation] >= tmp_flux_ratio

        if within_upper_limit and within_lower_limit:
            text_file.write(dataset_name + " " + tmp_key + "\n")
            num_exp+=1
            do_use.append(tmp_key)
            continue
            
        don_t_use.append(tmp_key)

# For S4

In [66]:
selection_file = "/fast/mbonse/s4/70_results/07_partial_contrast_grid/01_s4_HD22049_303_199_C-0065_C.txt"
#selection_file = "/fast/mbonse/s4/70_results/07_partial_contrast_grid/02_s4_HD22049_303_199_C-0065_C.txt"


with open(selection_file, "w") as text_file:
    num_exp = 0
    for tmp_key, values in contrast_instance.experimental_setups.items():
        for setup in [0, 1, 2]:

            if tmp_key == "0000":
                text_file.write(dataset_name + " " + tmp_key + " " + str(setup) + "\n")
                num_exp+=1
                continue

            tmp_separation = np.round(values["separation"] / 3.6, 1)
            tmp_flux_ratio = np.round(flux_ratio2mag(values["flux_ratio"]), 1)

            within_upper_limit = upper_limits[tmp_separation] <= tmp_flux_ratio
            within_lower_limit = lower_limits[tmp_separation] >= tmp_flux_ratio

            if within_upper_limit and within_lower_limit:
                text_file.write(dataset_name + " " + tmp_key + " " + str(setup) + "\n")
                num_exp+=1
                continue

In [68]:
num_exp

813

In [39]:
with open("/fast/mbonse/s4/70_results/07_partial_contrast_grid/01_first_test_s4_HD22049_303_199_C-0065_C.txt", "w") as text_file:
    num_exp = 0
    for tmp_key, values in contrast_instance.experimental_setups.items():
        for setup in [0, 1, 2]:
            if tmp_key not in ["0083a", "0105c", "0149b", "0041b", "0096c"]:
                continue

            tmp_separation = np.round(values["separation"] / 3.6, 1)
            tmp_flux_ratio = np.round(flux_ratio2mag(values["flux_ratio"]), 1)

            within_upper_limit = upper_limits[tmp_separation] <= tmp_flux_ratio
            within_lower_limit = lower_limits[tmp_separation] >= tmp_flux_ratio

            if within_upper_limit and within_lower_limit:
                text_file.write(dataset_name + " " + tmp_key + " " + str(setup) + "\n")
                continue

In [77]:
with open("/fast/mbonse/s4/70_results/07_partial_contrast_grid/02_first_test_s4_HD22049_303_199_C-0065_C.txt", "w") as text_file:
    num_exp = 0
    for tmp_key, values in contrast_instance.experimental_setups.items():
        if tmp_key not in ["0083a", "0105c", "0149b", "0041b", "0096c"]:
            continue

        tmp_separation = np.round(values["separation"] / 3.6, 1)
        tmp_flux_ratio = np.round(flux_ratio2mag(values["flux_ratio"]), 1)

        within_upper_limit = upper_limits[tmp_separation] <= tmp_flux_ratio
        within_lower_limit = lower_limits[tmp_separation] >= tmp_flux_ratio

        if within_upper_limit and within_lower_limit:
            text_file.write(dataset_name + " " + tmp_key + "\n")
            continue

## Create empty residuals for do not compute residuals

In [69]:
len(don_t_use)

240

In [70]:
from applefy.utils.file_handling import save_as_fits
from s4hci.detection_limits.applefy_wrapper import PCADataReductionGPU

In [71]:
pca_numbers = np.concatenate(
        [np.arange(0, 20, 2)[1:],
         np.arange(20, 50, 5),
         np.arange(50, 100, 10),
         np.arange(100, 200, 20),
         np.arange(200, 550, 50)])

pca_model = PCADataReductionGPU(
    approx_svd=10000,
    pca_numbers=pca_numbers,
    device=0,
    verbose=True)

In [72]:
dummy_image = np.ones_like(science_data[0])

In [73]:
method_names = pca_model.get_method_keys()
method_names.append("cADI")

In [75]:
method_names = ["s4_mean_raw_model", "s4_mean_planet_learned", "s4_mean_planet_learned_fine_tune", 
                "s4_median_raw_model", "s4_median_planet_learned", "s4_median_planet_learned_fine_tune"]

In [76]:
for tmp_method_name in method_names:
    print(tmp_method_name)
    for tmp_exp_id in don_t_use:
        tmp_file = tmp_exp_root / Path("residuals") \
            / Path(tmp_method_name) / Path("residual_ID_" + tmp_exp_id + ".fits")
        
        if not tmp_file.is_file():            
            save_as_fits(dummy_image, str(tmp_file))

s4_mean_raw_model
s4_mean_planet_learned
s4_mean_planet_learned_fine_tune
s4_median_raw_model
s4_median_planet_learned
s4_median_planet_learned_fine_tune


# DELETE ALL FILES
for tmp_method_name in method_names:
    print(tmp_method_name)
    for tmp_exp_id in don_t_use:
        tmp_file = tmp_exp_root / Path("residuals") \
            / Path(tmp_method_name) / Path("residual_ID_" + tmp_exp_id + ".fits")
        
        if tmp_file.is_file():            
            tmp_file.unlink()